In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen as uReq

print(os.listdir("./input"))

In [ ]:
# Data Extraction and exploration

fig_data = pd.read_csv("./input/fig_data_topicmodel.csv")

df=pd.DataFrame(fig_data)

In [ ]:
!pip install gensim
!pip install spacy

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')

import spacy

!python -m spacy download fr


In [ ]:
#tokenise words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations



In [ ]:
#create stop words list

stop_words = set(stopwords.words('french'))

#add words that aren't in the NLTK stopwords list
new_stopwords = ['cette', 'les', 'cet']
new_stopwords_list = stop_words.union(new_stopwords)

In [ ]:
#function to get words from the url
def web_scrape(url):
    #data = uReq(url)  # grabs the page
    # here we define the headers for the request
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
    webText=""
    try:
        # this request object will integrate your URL and the headers defined above
        req = urllib.request.Request(url=url, headers=headers)
    
        # calling urlopen this way will automatically handle closing the request
        with urllib.request.urlopen(req) as response:
            html = BeautifulSoup(response.read(), 'html.parser')
        #print(html)
        sent=[]
        for text in html.find_all('meta'):
                sent.append(str(text.get('content')))
        #print(sent)
        docs = list(sent)
        
        tfidf = TfidfVectorizer()
        tfidf.fit(docs)
        webText=tfidf.get_feature_names()
    except:
        print("Not able to open url",url)
    return webText

In [ ]:
# loop through the data frame, web scrape for each url and add scraped text

url=""
df["web_scrape"]=""
for idx, row in df.iterrows():
    if not(df.loc[idx,'url_cleaned'].find("http")):
        url=df.loc[idx,'url_cleaned']
    else:
        url="http://"+df.loc[idx,'url_cleaned']
    #print(url)
    text=web_scrape(url)
    df.loc[idx,'web_scrape']=" ".join(text)
    

In [ ]:
df.head(5)

In [ ]:
data_text = df['url_cleaned']+" "+df['web_scrape']
#data_text['url_cleaned']+=df['web_scrape']
data_text['index'] = data_text.index
documents = data_text

In [ ]:
print(len(documents))
print(documents[:5])

In [ ]:
data_words = list(sent_to_words(documents))

print(data_words[:1])

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words and len(word) > 3] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('fr', disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_words_bigrams[:1])

In [ ]:
#key is the word and value is the number of times that word occurs in the entire corpus
dictionary = gensim.corpora.Dictionary(data_lemmatized)
print(len(dictionary))

In [ ]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 20% of all documents
'''
dictionary.filter_extremes(no_below=3, no_above=0.20, keep_n= 100000)
print(len(dictionary))

In [ ]:
# Create Dictionary
#id2word = corpora.Dictionary(dictionary)

# Create Corpus
#bag of words
corpus = [dictionary.doc2bow(doc) for doc in data_lemmatized]

# View
print(corpus[:1])

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)